In [1]:
import tifffile
import pandas as pd
import joblib as jb
import numpy as np
import matplotlib.pyplot as plt
import ee
import geemap
from pathlib import Path

In [10]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [11]:
collection = ee.ImageCollection("COPERNICUS/S2_SR")

In [12]:
feature = Map.draw_last_feature
roi = feature.geometry()

In [23]:
Map.addLayer(collection_roi, {
  'bands': ['B1'], 
  'min': 0,
  'max': 255,
});

In [15]:
out_dir = '/home/vinicius/Área de Trabalho/Faculdade/2020_1/TCC/download_data/new_sentinel_data/'

In [16]:
collection_roi = collection.filterBounds(roi).filterDate('2018-01-01', '2020-01-01').filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',4)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])

In [17]:
collection_roi.first().getInfo()['bands']

[{'id': 'B1',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [1830, 1830],
  'crs': 'EPSG:32722',
  'crs_transform': [60, 0, 699960, 0, -60, 7100020]},
 {'id': 'B2',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [10980, 10980],
  'crs': 'EPSG:32722',
  'crs_transform': [10, 0, 699960, 0, -10, 7100020]},
 {'id': 'B3',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [10980, 10980],
  'crs': 'EPSG:32722',
  'crs_transform': [10, 0, 699960, 0, -10, 7100020]},
 {'id': 'B4',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [10980, 10980],
  'crs': 'EPSG:32722',
  'crs_transform': [10, 0, 699960, 0, -10, 7100020]},
 {'id': 'B5',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [5490, 5490],
  'crs': 'E

In [11]:
count = collection_roi.size()
print('Count: ', str(count.getInfo())+'\n')

Count:  14



In [20]:
geemap.ee_export_image_collection(collection_roi, out_dir=out_dir, scale=25, file_per_band=True, region=roi)

Total number of images: 14

Exporting 1/14: 20190130T132239_20190130T132233_T22JGR.tif
Generating URL ...
Please wait ...
Data downloaded to /home/vinicius/Área de Trabalho/Faculdade/2020_1/TCC/download_data/new_sentinel_data


Exporting 2/14: 20190130T132239_20190130T132233_T22JGS.tif
Generating URL ...
Please wait ...
Data downloaded to /home/vinicius/Área de Trabalho/Faculdade/2020_1/TCC/download_data/new_sentinel_data


Exporting 3/14: 20190209T132239_20190209T132233_T22JGS.tif
Generating URL ...
Please wait ...
Data downloaded to /home/vinicius/Área de Trabalho/Faculdade/2020_1/TCC/download_data/new_sentinel_data


Exporting 4/14: 20190415T132241_20190415T132235_T22JGR.tif
Generating URL ...
Please wait ...
Data downloaded to /home/vinicius/Área de Trabalho/Faculdade/2020_1/TCC/download_data/new_sentinel_data


Exporting 5/14: 20190415T132241_20190415T132235_T22JGS.tif
Generating URL ...
Please wait ...
Data downloaded to /home/vinicius/Área de Trabalho/Faculdade/2020_1/TCC/downlo